# Caching, Session Storage, and Search Workshop
## Hands-on workshop

![diagram of session storage on external DB](img/SessionStore.png)

Presented to Discover Financial Services, August 13, 2025

This environment already has a Redis database up and running. It's currently empty, and you can see the DB and the progress we make going through the lab by using the **INSIGHT** link on the starting page. (Where you initially clicked **IDE**.)

![picture of the buttons on the starting screen](img/insight.png)

We'll start by loading some sample data into our DB so we have somewhere to search.  
Run the next code snippet to populate the DB.

In [ ]:
%run load_restaurants

import load_sessions 
from load_sessions import LoadSessions 

#import importlib
#importlib.reload(load_sessions)

load_sessions.LoadSessions.load_premade()
load_sessions.LoadSessions.load_session(25)


### GeoSearch

Even before we get to the search and query functions, we can find locations.

The following search does **not** use secondary indexing. We put a few downtown restaurants in a GeoSet.  
This is a sorted set and each member has a score that corresponds to it's geolocation. Queries are made using longitude and latitude *or* geohash values. We'll keep it easy and use lon, lat.

![Ogilve Transport Center upper level](img/256px-Ogilvie_Transportation_Center.jpg)
Let's say a customer has just arrived at Ogilve Transportation Center and they are hungry. So they get out their smart phone and search for a sit-down restaurant nearby. Say the customer is roughly mid-lobby at longitude -87.640482 and latitude 41.882414.

Remember, this search **isn't** a search - well not using the search and query function. We're just going to look through the set of downtown restaurants. So the syntax is `GEOSEARCH` rather than `FT.SEARCH` you saw in the earlier presentation.

So the customer opens their app and searches for a nice restaurant. The syntax is as follows for a search by radius. (The alternative is to search within a rectangle `BYBOX`.)

```
GEOSEARCH key FROMLONLAT longitude latitude BYRADIUS radius units
```

Optionally, you can also request `ASC`ending or `DESC`ending, `WITHCOORD`inates and/or `WITHDIST`ance

In [ ]:
#Here is the starting code.
from redis_connection import RedisConnection
redis = RedisConnection.get_client()

#the geoset is named chicago_restaurants
key_name = 'chicago_restaurants'
# feel free to change the radius; decimal values are OK
rad = 1
# feel free to change the units. The only options are unit = 'mi' mile, 'km' kilometer, 'ft' feet, or 'm' meters.
units = 'mi'

redis.geosearch(key_name, longitude=-87.640482, latitude=41.882414, radius=rad, unit=units, withdist=True, withcoord=True)

### Now let's tackle *real* search.

As previously discussed, you need to analyze your data to be able to develop a meaningful search shema.

Let's say our session data looks something like this:


```json
{
    "lastAccessed":1723574040,
    "creation":1723572000,
    "user": {
        "firstname":"Paul",
        "lastname":"Bunion"
        },
    "visited":["www.megashop.com","www.whatever.org","www.progressive.com"], 
    "location": "-87.6359,41.8788", 
    "cart":[
        {
            "itemId":"bigaxe",
            "itemCost":918.99,
            "quantity":2
        }
        {
            "itemID":"oxchow-ton",
            "itemCost":"2449.99"
            "quantity":1
        }
    ]
}
```

Since the session data is in JSON, we might as well store the document in JSON too. So this is the syntax:

```redis
FT.CREATE key_name ON JSON PREFIX count prefix

SCHEMA
json.path.name AS alias type
next field...
```

For example:

```redis
FT.CREATE session:index ON JSON PREFIX 1 "session:"
SCHEMA
$.lastAccessed as lastAccessed NUMERIC SORTABLE
$.user.firstname as first TAG
$.user.lastname as last TAG
$.visited as visited TEXT
$.cart[*].itemID as itemid TAG
$.location as location GEO
```

In [ ]:
# answer in challenge1.py
from redis.commands.search.field import GeoField, NumericField, TextField, TagField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query, NumericFilter

# This presumes that "from redis_connection import RedisConnection" and "redis = RedisConnection.get_client()" have previously been run. We did that in the previous example.

# >>> START CODING CHALLENGE <<<
index_name = '' # give your index a name

redis.ft(index_name).create_index(
    (
        NumericField("$.lastAccessed", as_name="lastAccessed", sortable=True),
        TagField("$.user.firstname", as_name="first"),
        # add last name
        # add visited
        TagField("$.cart[*].itemID", as_name="itemid"),
        # GeoField(...) # add location
    ),
    definition=IndexDefinition(prefix=["session:"], index_type=IndexType.JSON))
# >>> END CODING CHALLENGE <<<

In [ ]:
# This sample code searches for all sessions within 1 km of the entrance to OTC. 
# This example uses hard coded values but it's no big deal to receive all the parameters interactively
# lon = -87.640482, lat = 41.882414, radius = 100, unit = 'm'

index_name = 'session:index' # Change this to whatever index name you used.

# the CLI syntax is "@location:[lon lat radius unit]" 
query = '@location:[-87.640482 41.882414 100 m]'
# feel free to try other values, but all our sessions are downtown Chicago, so there's that.

result = redis.ft(index_name).search(query)

print(result)

### Here are CLI samples for the syntax to more useful searches.

You can test these examples (and much more) on the command line in Redis Insight, and/or by connecting to your Redis DB from the terminal in this IDE.

```redis
FT.SEARCH session:index '@visited:("redis.io")' RETURN 1 $.visited

FT.SEARCH session:index '@visited:("redis.io") -@visited:("www.example.com")' RETURN 1 $.visited

FT.SEARCH session:index '@itemid:{roncowidget}' RETURN 0

FT.SEARCH session:index '@itemid:{MacBook | roncowidget}' RETURN 0
```

More hypothetical examples (in python)

```python
#  sort by a sortable field in ascending order 
# (you can sort by a non-sortable field but it will be slower)
redis.ft(index_name).search(Query('some_query').sort_by(field='length', asc=True))

#  search a specific field
#  any TEXT or TAG search can have wild cards  
redis.ft(index_name).search(Query('@visited:*progressive*'))

#  search specific fields - OR operation (name OR type matches on '*idget')
redis.ft(index_name).search(Query('@name|type:*idget'))

#  search specific fields - AND operation (space separated = AND) 
redis.ft(index_name).search(Query('@name:whatever @type:*actu*'))
```

In [ ]:
# Coding challenge 2
# Now let's put that into practice.
# Find the sessions that have either a superwidget or an okwidget in the shopping cart
# AND it has been active since 1:15 pm on our hypothetical day (this Unix timestamp: 1723576500).
# the special numeric value +inf (without any quotes) means unbounded on the upper end.

index_name = 'session:index' 

# >>> START CODING CHALLENGE <<<

# >>> END CODING CHALLENGE <<<

print(result)


In [ ]:
# Extra credit: play with additional criteria such as location, sites visited, wild card searches etc.